### После изучения основ предлагаем сделать свой пайплайн самостоятельно. Попробуем на практике реализовать пайплайн, добавить в него преобразования, изменить параметры. Приступим!

In [9]:
# загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
 
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [10]:
df = pd.read_csv('data/Red.zip')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


In [12]:
df

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
...,...,...,...,...,...,...,...,...
8661,6th Sense Syrah 2016,United States,Lodi,Michael David Winery,3.8,994,16.47,2016
8662,Botrosecco Maremma Toscana 2016,Italy,Maremma Toscana,Le Mortelle,4.0,995,20.09,2016
8663,Haut-Médoc 2010,France,Haut-Médoc,Château Cambon La Pelouse,3.7,996,23.95,2010
8664,Shiraz 2019,Australia,South Eastern Australia,Yellow Tail,3.5,998,6.21,2019


In [13]:
df.describe(include='object')

,Name,Country,Region,Winery,Year
count,8666,8666,8666,8666,8666
unique,6721,30,624,2714,32
top,Cabernet Sauvignon 2017,Italy,Rioja,Errazuriz,2016
freq,43,2650,325,40,1776


In [14]:
TAGRET_FEATURE = 'Price'

### Задание 6.1
Предлагаем использовать датасет, с которым вы работали при создании пайплайна (файл Red.csv ).

Вам следует выполнить следующее:

Добавить обработку столбца 'Region' в пайплайн, полученный ранее в модуле, с использованием OrdinalEncoder.
Важно! Для совпадения результатов процесс трансформации столбцов должен выполняться в следующей последовательности:
* Кодирование столбца 'Region'.
* Стандартизация столбца 'Price'.
* Кодирование столбца 'Country'.

In [15]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer

In [16]:
ct = ColumnTransformer([
    ('ordinal', OrdinalEncoder(), ['Region']),
    ('scaler', StandardScaler(), ['Price']),
    ('one_hot', OneHotEncoder(), ['Country'])
])

print(ct)

ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(), ['Region']),
                                ('scaler', StandardScaler(), ['Price']),
                                ('one_hot', OneHotEncoder(), ['Country'])])


In [17]:
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor())])

In [18]:
X, y = df[['Region', 'Price', 'Country']], df['Rating']

In [19]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor())])

In [20]:
import joblib

In [21]:
joblib.dump(pipeline, 'pipeline_wine.pkl')

['pipeline_wine.pkl']

In [22]:
def rmse(y, y_pred):
    return mean_squared_error(y, y_pred, squared = False)

In [23]:
y_pred = pipeline.predict(X)
print(f'Качество по метрике R2: {r2_score(y, y_pred):.4f}')
print(f'Качество по RSME: {rmse(y, y_pred):.4f}')

Качество по метрике R2: 0.9226
Качество по RSME: 0.0859


In [24]:
df_test = pd.read_csv('data\Red_test.zip')

In [25]:
X_test, y_test = df_test[['Region', 'Price', 'Country']], df_test['Rating']

In [26]:
y_test_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: {r2_score(y_test, y_test_pred):.4f}')
print(f'Качество по RSME: {rmse(y_test, y_test_pred):.4f}')

Качество по метрике R2: 0.9364
Качество по RSME: 0.0783


### Задание 6.2
Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [27]:
pipeline_loaded = joblib.load('pipeline_wine.pkl')

In [32]:
pipeline_loaded.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(n_estimators=200))])>

In [30]:
pipeline_loaded.set_params(rf__n_estimators=200)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(n_estimators=200))])

In [33]:
pipeline_loaded.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(n_estimators=200))])>

In [51]:
pipeline_loaded.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(n_estimators=200))])

In [52]:
y_test_pred = pipeline_loaded.predict(X_test)
print(f'Качество по метрике R2: {r2_score(y_test, y_test_pred):.4f}')
print(f'Качество по RSME: {rmse(y_test, y_test_pred):.4f}')

Качество по метрике R2: 0.9363
Качество по RSME: 0.0783


### Задание 6.3
Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:
* Собрать StackingRegressor:
* В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
* В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
* Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).
* Зафиксировать random_state=42 (для всех моделей).
* Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
* Обучить модель на тренировочной выборке.

In [53]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

In [54]:
random_state=42

# Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('ridge', RidgeCV()),
    ('dt', DecisionTreeRegressor(random_state=random_state))
]

# Создаем объект класса стекинг
st_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10, random_state=random_state)
)

In [55]:
pipeline_stacking = Pipeline([('ct', ct), ('sr', st_reg)])
pipeline_stacking.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(),
                                                  ['Region']),
                                                 ('scaler', StandardScaler(),
                                                  ['Price']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['Country'])])),
                ('sr',
                 StackingRegressor(estimators=[('ridge', RidgeCV()),
                                               ('dt',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         random_state=42)))])

In [56]:
y_test_pred = pipeline_stacking.predict(X_test)
print(f'Качество по метрике R2: {r2_score(y_test, y_test_pred):.4f}')
print(f'Качество по RSME: {rmse(y_test, y_test_pred):.2f}')

Качество по метрике R2: 0.6505
Качество по RSME: 0.18
